<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_base_pretrain_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This trains a roberta-base model on 100 epochs of TAPT helpfulness

At epoch 20 was eval score of 1.472466

# Settings
* Set batch size to 21, gradient_accumulation_steps=2, ~6 reviews per row = 252 batch size (paper has 256)
* LR 0.0001
* 100 epochs
* torch_compile=True (Because no adapter)
* early_stopping_patience=10
* no warmup







# Setup

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
# !pip install adapters
# !pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [8]:
from transformers import AutoConfig, AutoModelForMaskedLM
pretrain_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [11]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_base_pretraining_model_final",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [12]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [13]:
from transformers import Trainer

pretrain_trainer = Trainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
0,1.769700,1.590389
2,1.631400,1.546102
4,1.576600,1.531151
6,1.527100,1.503400
8,1.490600,1.494203
10,1.463800,1.490043
12,1.428500,1.469219
14,1.405100,1.479018
16,1.383200,1.472989
18,1.361700,1.462458


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=11160, training_loss=1.3398856296334216, metrics={'train_runtime': 5460.2935, 'train_samples_per_second': 178.635, 'train_steps_per_second': 4.249, 'total_flos': 1.2325878379148083e+17, 'train_loss': 1.3398856296334216, 'epoch': 48.0})


In [15]:
import pandas as pd

pd.DataFrame(pretrain_trainer.state.log_history)

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.7697,4.176953,0.000099,1.0,232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,232,1.590389,9.2981,212.947,10.217,NaN,NaN,NaN,NaN,NaN
2,1.6633,4.389059,0.000098,2.0,465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2.0,465,1.565005,8.8642,223.370,10.717,NaN,NaN,NaN,NaN,NaN
4,1.6314,3.624543,0.000097,3.0,697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1.1490,2.966422,0.000053,47.0,10927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,47.0,10927,1.449236,8.7318,226.757,10.880,NaN,NaN,NaN,NaN,NaN
94,1.1401,3.858835,0.000052,48.0,11160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,48.0,11160,1.450196,8.7402,226.540,10.869,NaN,NaN,NaN,NaN,NaN


In [16]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 1.4543
{'eval_loss': 1.4543291330337524, 'eval_runtime': 8.8182, 'eval_samples_per_second': 224.536, 'eval_steps_per_second': 10.773, 'epoch': 48.0}


In [17]:
pretrain_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1713288685.f412fc789800.2047.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

events.out.tfevents.1713283190.f412fc789800.2047.0:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/tapt_helpfulness_base_pretraining_model_final/commit/fdc29596c6241669909a3f2a67e846f52c90db62', commit_message='End of training', commit_description='', oid='fdc29596c6241669909a3f2a67e846f52c90db62', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 94 minutes OR 5615 seconds


# Classification

In [19]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
